## **Deskripsi Tugas**
Lakukan analisis, desain, dan implementasi algoritma Genetic algorithm (GA) ke dalam suatu program komputer untuk menemukan **nilai maximum** dari fungsi: 

$$h(x,y) = (cos x^2 + sin y^2) + (x + y)$$

dengan batasan $-1 \leq x \leq 2$ dan $-1 \leq y \leq 1$.

## **Library yang Digunakan**

1. `random`, untuk mengenerate populasi dan bilangan acak.
2. `math`, untuk kalkulasi h(x,y) menggunakan fungsi sin dan cos.
3. `heapq`, untuk memilih $n$ elemen terbaik dari suatu list.

In [1]:
import random
from math import sin, cos
import heapq

## **Fungsi Bantuan**

In [2]:
def revBit(bit):
  return '0' if bit == '1' else '1'

## **Solusi Tugas**

### Informasi Soal

In [3]:
def h(x, y):
  return (cos(x**2) * sin(y**2)) + (x + y)

batas_x = [-1, 2]
batas_y = [-1, 1]

### Desain Kromosom

Disini kami representasikan sebagai biner dengan banyak bit x = 50 dan banyak bit y = 50.

In [4]:
len_x = len_y = 50

def generateRandomChromosome():
  # Generate string biner acak dengan ukuran len_x + len_y
  hasil = ""
  for i in range(len_x + len_y):
    hasil += str(random.randint(0, 1))
  return hasil

def decodeGenotipe(g, t):
  # pakai rumus di slide
  n = len_x if t == "x" else len_y
  twos = sum([2**-i for i in range(1, n+1)])
  jmlh = sum([int(g[i-1])*(2**-i) for i in range(1, n+1)])

  if (t == "x"):
    return batas_x[0] + (batas_x[1] - batas_x[0]) / twos * jmlh

  return batas_y[0] + (batas_y[1] - batas_y[0]) / twos * jmlh

def toFenotipe(c):
  strX, strY = c[:len_x], c[len_y:]
  return decodeGenotipe(strX, "x"), decodeGenotipe(strY, "y")

def genRandomPopulation(pop_size):
  popu = []
  for i in range(pop_size):
    popu.append(generateRandomChromosome())
  return popu


### Pemilihan Orangtua

Disini kami menggunakan mekanisme **roulette wheel** yang dibagi secara proporsional berdasarkan nilai fitness. Orangtua akan dipilih secara random dan yang memiliki proporsi tinggi akan semakin mungkin untuk terpilih.

In [5]:
def roulette_wheel(prob_dist):
  # prob_dist adalah list of probabilities yang jika dijumlahkan akan = 1.
  # Fungsi ini mengembalikan idx orang tua yang terpilih.
  panah_wheel = random.random()

  sumSejauhIni = 0
  idx = 0
  for i in range(len(prob_dist)):
    sumSejauhIni += prob_dist[i]
    if sumSejauhIni >= panah_wheel:
      break
    idx += 1
  return idx

### Crossover
Crossover dengan metode Uniform Crossover yang berkemungkinan $P_c$ akan terjadi.

In [6]:
def crossover(c1, c2, p_c):
  crossoverTerjadi = random.random()
  if (crossoverTerjadi <= p_c):
    hasil = ""
    for bit1, bit2 in zip(c1, c2):
      pilih_acak = random.random()
      hasil += bit1 if (pilih_acak <= 0.5) else bit2
    return hasil
  return c1

### Mutasi

Mutasi dengan kemungkinan $P_m$ akan terjadi untuk setiap bit kromosom.

In [7]:
def mutation(c, p_m):
  hasil = ""
  for bit in c:
    real_acak = random.random()
    hasil += revBit(bit) if (real_acak <= p_m) else bit
  return hasil

### Pergantian Generasi

Pergantian generasi dilakukan dengan cara mengambil $n$ individu terbaik dari gabungan orangtua dan anaknya.

In [8]:
def replaceGeneration(parent, parent_fit, child, child_fit, pop_size):
  parent = list(zip(parent, parent_fit))
  child = list(zip(child, child_fit))

  new_gen = heapq.nlargest(pop_size, parent + child, key=lambda x:x[1])
  return [new_gen[i][0] for i in range(pop_size)]

### Genetic Algorithm

In [9]:
def geneticAlgorithm(pop, pop_size, num_of_gen, p_c, p_m, fitness):
  # parameter : first gen, ukuran populasi, berapa kali generasi, p_c, p_m, fungsi fitness

  for i in range(num_of_gen):
    pop_fit = [fitness(*toFenotipe(pop[i])) for i in range(pop_size)]
    print("Fitness terbaik pada generasi ke-{0}\t: {1}".format(i+1, max(pop_fit)))
    sum_fit = sum(pop_fit)
    prob_dist = [pop_fit[i]/sum_fit for i in range(pop_size)]

    new_pop = []
    for j in range(pop_size):
      # Pemilihan orangtua
      parent_1 = pop[roulette_wheel(prob_dist)]
      parent_2 = pop[roulette_wheel(prob_dist)]

      # Crossover
      co_result = crossover(parent_1, parent_2, p_c)

      # Mutation
      mu_result = mutation(co_result, p_m)

      new_pop.append(mu_result)
    new_pop_fit = [fitness(*toFenotipe(new_pop[i])) for i in range(pop_size)]

    # Pergantian generasi
    pop = replaceGeneration(pop, pop_fit, new_pop, new_pop_fit, pop_size)
  
  pop_fit = [fitness(*toFenotipe(pop[i])) for i in range(pop_size)]
  return max(zip(pop, pop_fit), key=lambda x:x[1])

  

## **Main Program**

In [10]:
# Parameter GA
N = 500
num_of_gen = 100
p_c = 0.9
p_m = 1/(len_x + len_y)

# Inisialisai populasi
pop = genRandomPopulation(N)

# Algoritma Genetika
chromosomeHasil, fitHasil = geneticAlgorithm(pop, N, num_of_gen, p_c, p_m, h)

# Tampilkan hasil
print("\n======== Hasil Terbaik ========")
print("Chromosome x\t= ", chromosomeHasil[:len_x])
print("Chromosome y\t= ", chromosomeHasil[len_y:])
print("(x , y)\t\t= ({0} , {1})".format(*toFenotipe(chromosomeHasil)))
print("Fitness\t\t= ", fitHasil)

Fitness terbaik pada generasi ke-1	: 2.443213610483161
Fitness terbaik pada generasi ke-2	: 2.4706342888920365
Fitness terbaik pada generasi ke-3	: 2.4706342888920365
Fitness terbaik pada generasi ke-4	: 2.4706342888920365
Fitness terbaik pada generasi ke-5	: 2.4787721776398324
Fitness terbaik pada generasi ke-6	: 2.4787721776398324
Fitness terbaik pada generasi ke-7	: 2.4787721776398324
Fitness terbaik pada generasi ke-8	: 2.4787721776398324
Fitness terbaik pada generasi ke-9	: 2.4794000115540156
Fitness terbaik pada generasi ke-10	: 2.4806744843100215
Fitness terbaik pada generasi ke-11	: 2.4807952371430644
Fitness terbaik pada generasi ke-12	: 2.4812305600507556
Fitness terbaik pada generasi ke-13	: 2.4813713989200226
Fitness terbaik pada generasi ke-14	: 2.4815076933188425
Fitness terbaik pada generasi ke-15	: 2.4815707491424424
Fitness terbaik pada generasi ke-16	: 2.4815707491424424
Fitness terbaik pada generasi ke-17	: 2.481587121976278
Fitness terbaik pada generasi ke-18	: 2.48